# Structured output

OpenAI offers functionality for defining a structure of the messages generated by LLMs, AG2 enables this functionality by propagating `response_format`, in the LLM configuration for your agents, to the underlying client. This is currently only supported by OpenAI.

For more info on structured output, please check [here](https://platform.openai.com/docs/guides/structured-outputs)


````{=mdx}
:::info Requirements
Install `ag2`:
```bash
pip install ag2
```

> **Note:** If you have been using `autogen` or `pyautogen`, all you need to do is upgrade it using:  
> ```bash
> pip install -U autogen
> ```
> or  
> ```bash
> pip install -U pyautogen
> ```
> as `pyautogen`, `autogen`, and `ag2` are aliases for the same PyPI package.  


For more information, please refer to the [installation guide](/docs/user-guide/basic-concepts/installing-ag2).
:::
````

### Supported clients
AG2 has structured output support for following client providers:
- OpenAI (`openai`)
- Anthropic (`anthropic`)
- Google Gemini (`google`)
- Ollama (`ollama`)

## Set your API Endpoint

The [`config_list_from_json`](https://docs.ag2.ai/docs/api-reference/autogen/config_list_from_json#config-list-from-json) function loads a list of configurations from an environment variable or a json file. Structured Output is supported by OpenAI's models from gpt-4-0613 and gpt-3.5-turbo-0613.

In [1]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "tags": ["gpt-4o-mini-response-format"],
    },
)

/Users/tvrtko/Documents/projects/AG2/ag2/.venv/lib/python3.11/site-packages/flaml/__init__.py:20: UserWarning: flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.
  warnings.warn("flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.")


In [2]:
config_list[0]["response_format"]

{'$defs': {'Step': {'properties': {'explanation': {'title': 'Explanation',
     'type': 'string'},
    'output': {'title': 'Output', 'type': 'string'}},
   'required': ['explanation', 'output'],
   'title': 'Step',
   'type': 'object'}},
 'properties': {'steps': {'items': {'$ref': '#/$defs/Step'},
   'title': 'Steps',
   'type': 'array'},
  'final_answer': {'title': 'Final Answer', 'type': 'string'}},
 'required': ['steps', 'final_answer'],
 'title': 'MathReasoning',
 'type': 'object'}

In [3]:
from datamodel_code_generator import generate

In [4]:
assert False

AssertionError: 

````{=mdx}
:::tip
Learn more about configuring LLMs for agents [here](/docs/topics/llm_configuration).
:::
````

## Example: math reasoning

Using structured output, we can enforce chain-of-thought reasoning in the model to output an answer in a structured, step-by-step way.

### Define chat actors

Now we can define the agents that will solve the posed math problem. 
We will keep this example simple; we will use a `UserProxyAgent` to input the math problem and an `AssistantAgent` to solve it.

The `AssistantAgent` will be constrained to solving the math problem step-by-step by using the `MathReasoning` response format we defined above.

The `response_format` is added to the LLM configuration and then this configuration is applied to the agent.

In [ ]:
llm_config = {
    "config_list": config_list,
    "cache_seed": 42,
}

user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    human_input_mode="NEVER",
)

assistant = autogen.AssistantAgent(
    name="Math_solver",
    llm_config=llm_config,  # Response Format is in the configuration
)

### Start the chat

Let's now start the chat and prompt the assistant to solve a simple equation. The assistant agent should return a response solving the equation using a step-by-step `MathReasoning` model.

In [ ]:
user_proxy.initiate_chat(assistant, message="how can I solve 8x + 7 = -23", max_turns=1, summary_method="last_msg")